## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [ ]:
# enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

## 2.1 Configure the endpoint

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 3.1 Configure the deployment

In [ ]:
# create a myModel deployment
model_version=1

model = Model(
    path="sklearn-diabetes/model",
    type=AssetTypes.CUSTOM,
    version=model_version,
    description="my sample mlflow model",
)

## Register Model
ml_client.models.create_or_update(model)

blue_deployment = ManagedOnlineDeployment(
    name="myModel",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
# myModel deployment takes 100 traffic
endpoint.traffic = {"myModel": 100}
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)